In [ ]:
!nvidia-smi

Thu Aug  3 21:04:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Clone repository

In [ ]:
!git clone https://github.com/kaylode/vehicle-counting.git main
%cd main

Cloning into 'main'...
remote: Enumerating objects: 1036, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1036 (delta 23), reused 21 (delta 19), pack-reused 992
Receiving objects: 100% (1036/1036), 263.57 MiB | 23.41 MiB/s, done.
Resolving deltas: 100% (513/513), done.
/content/main


In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
!pip uninstall opencv-python-headless
!pip install opencv-python-headless==4.5.4.60

Found existing installation: opencv-python-headless 4.8.0.74
Uninstalling opencv-python-headless-4.8.0.74:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/cv2/*
    /usr/local/lib/python3.10/dist-packages/opencv_python_headless-4.8.0.74.dist-info/*
    /usr/local/lib/python3.10/dist-packages/opencv_python_headless.libs/libavcodec-402e4b05.so.59.37.100
    /usr/local/lib/python3.10/dist-packages/opencv_python_headless.libs/libavformat-0e2a2001.so.59.27.100
    /usr/local/lib/python3.10/dist-packages/opencv_python_headless.libs/libavutil-ac9b7a1d.so.57.28.100
    /usr/local/lib/python3.10/dist-packages/opencv_python_headless.libs/libcrypto-4b97533b.so.1.1
    /usr/local/lib/python3.10/dist-packages/opencv_python_headless.libs/libgfortran-91cc3cb1.so.3.0.0
    /usr/local/lib/python3.10/dist-packages/opencv_python_headless.libs/libopenblas-r0-f650aae0.3.3.so
    /usr/local/lib/python3.10/dist-packages/opencv_python_headless.libs/libpng16-7379b3c3.so.16.40.0
    /usr/local/lib/p

In [ ]:
!git reset --hard HEAD
!git pull

# Copy video file
For input data, you need 2 file:
- Video
- Directions and Zone annotation file associated with the street in the video, have same name with the video


# Start inference
Define these things before run:
- Path to street annotation files in **configs/cam_configs.yaml**
- Path to video file
- Model's checkpoint

In [ ]:
!gdown https://drive.google.com/file/d/1-Y6H3QdRevfBKYDQxgRiR2CRinRVPt9O -O /content/yolov5s.pt

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:28: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1-Y6H3QdRevfBKYDQxgRiR2CRinRVPt9O
To: /content/yolov5s.pt
77.6kB [00:00, 4.34MB/s]


In [ ]:
!gdown https://drive.google.com/file/d/10It3-bByVQUiLV9q4sdJDXQ3bNK9obKi -O /content/yolov5m.pt

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:28: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/10It3-bByVQUiLV9q4sdJDXQ3bNK9obKi
To: /content/yolov5m.pt
77.6kB [00:00, 2.89MB/s]


In [ ]:
!mkdir /content/results

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

In [ ]:
%cd yolov5
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt

In [ ]:
%cd yolov5
!python train.py --img 640 --batch 16 --epochs 3 --data VisDrone.yaml --weights yolov5s.pt

In [ ]:
%cd /content/main
!python run.py --input_path='/content/main/demo/sample/cam_04.mp4' \
              --output_path="/content/results" \
              --weight="/content/yolov5m.pt"

/content/main
##########   CONFIGURATION INFO   ##########
model_name:	yolov5s
gpu_devices:	0
num_epochs:	300
batch_size:	16
num_workers:	2
min_iou:	0.45
min_conf:	0.25
max_det:	300
image_size:	[640, 640]
keep_ratio:	True


Nubmer of gpus: 1
CUDA:0 (Tesla T4, 15101.8125MB)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

YOLOv5 🚀 2023-7-28 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

100% 14.1M/

# Tracking results
There will be two result files:
- A **.csv** contains tracking result in every frames
- A **.mp4** plots the results from .csv to original video

In [ ]:
import pandas as pd
pd.read_csv('/content/results/cam_04.csv').head()

,track_id,frame_id,box,color,label,direction,fpoint,lpoint,fframe,lframe
0,2,3,"[856, 382, 883, 441]","(122, 150, 233)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31
1,2,4,"[866, 378, 892, 434]","(122, 150, 233)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31
2,2,5,"[869, 377, 894, 431]","(122, 150, 233)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31
3,2,6,"[869, 376, 895, 431]","(122, 150, 233)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31
4,2,7,"[872, 375, 897, 429]","(122, 150, 233)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31


In [ ]:
from google.colab import files
%cd /content/results
files.download('cam_04.mp4')

/content/results


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>